はじめに必要そうなモジュールをimportします．

今回は統計解析にScipyではなく，[Pingouin](https://pingouin-stats.org/build/html/index.html)を使ってみます．
installは
`pip install pingouin`
で可能です．

In [4]:
import scipy #科学計算用
import pandas as pd #データ分析用
import numpy as np #多次元配列計算用

import pingouin as pg #統計解析用

csvファイルを読み込ませてdataframeにします．
ファイルを検索するためのpathを通しておきます．

でも最初から同じフォルダもしくは子フォルダにあると便利ですよね．

In [7]:
import sys

sys.path.append(r'C:\Users\zucky\OneDrive - The University of Tokyo\行動実験解析')
sys.path

['c:\\Users\\zucky\\OneDrive\\デスクトップ\\test\\github-lab',
 'c:\\Users\\zucky\\anaconda3\\python311.zip',
 'c:\\Users\\zucky\\anaconda3\\DLLs',
 'c:\\Users\\zucky\\anaconda3\\Lib',
 'c:\\Users\\zucky\\anaconda3',
 '',
 'c:\\Users\\zucky\\anaconda3\\Lib\\site-packages',
 'c:\\Users\\zucky\\anaconda3\\Lib\\site-packages\\win32',
 'c:\\Users\\zucky\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'c:\\Users\\zucky\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\zucky\\OneDrive - The University of Tokyo\\行動実験解析',
 'C:\\Users\\zucky\\OneDrive - The University of Tokyo\\行動実験解析']

In [13]:
df =pd.read_csv('copy-231221_BLT_A19d_CsChrimson.csv')

df

,Unnamed: 0,bwd _no stim.(30s),bwd _during stim.(30s),fwd _no stim.(30s),fwd _during stim.(30s),turn _no stim.(30s),turn _stim.(30s),Unnamed: 7,Unnamed: 8,Unnamed: 9
0,R15C03>CsCh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0mW 1,0.0,1.0,17.0,19.0,6.0,2.0,NaN,NaN,tsh-Gai80×
2,5.0mW 2,0.0,0.0,26.0,17.0,3.0,2.0,NaN,0.58で壁に到達し静止，stim3中はプレート壁上を歩いていたためノーカン,NaN
3,5.0mW 3,0.0,0.0,18.0,15.0,1.0,2.0,NaN,"stim1,2でrolling",NaN
4,5.0mW 4,0.0,0.0,18.0,14.0,1.0,1.0,NaN,NaN,おそらくtsh-Gai80×
5,3.0mW 1,0.0,0.0,15.0,15.0,7.0,3.0,NaN,"0.53, 1.19でbwd1回",NaN
6,3.0mW 2,0.0,0.0,21.0,19.0,0.0,0.0,NaN,0.49でプレート壁に衝突,NaN
7,3.0mW 3,0.0,0.0,28.0,24.0,0.0,2.0,NaN,0.36で壁際まで進み静止．プレート壁に衝突,NaN
8,3.0mW 4,0.0,0.0,19.0,16.0,3.0,3.0,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


今回は条件の異なるデータが同じ列内に混在している悪いcsvファイルを使用したため，データの切り分け作業が発生します．違う実験のデータは違うファイルにまとめましょう(戒め)．

R72E03>CsCh，3.0mWのデータ(15-18行目)について検定をしていくことにします．

In [29]:
#dataframeをseriesに変換
ser_bwd_ctrl =df.loc[15:19, 'bwd _no stim.(30s)']
ser_bwd_exp =df.loc[15:19, 'bwd _during stim.(30s)']

ser_fwd_ctrl =df.loc[15:19, 'fwd _no stim.(30s)']
ser_fwd_exp =df.loc[15:19, 'fwd _during stim.(30s)']

ser_turn_ctrl =df.loc[15:19, 'turn _no stim.(30s)']
ser_turn_exp =df.loc[15:19, 'turn _stim.(30s)']

データが抽出できているか確認してみます．

In [33]:
ser_bwd_ctrl

15    0.0
16    0.0
17    0.0
18    0.0
Name: bwd _no stim.(30s), dtype: float64

In [32]:
print(ser_bwd_exp)
print(type(ser_bwd_exp))

15    0.0
16    0.0
17    2.0
18    0.0
Name: bwd _during stim.(30s), dtype: float64
<class 'pandas.core.series.Series'>


seriesのままでは統計にかけられないためlistに変換します．

In [34]:
ls_bwd_ctrl =ser_bwd_ctrl.tolist()
ls_bwd_exp =ser_bwd_exp.tolist()

ls_fwd_ctrl =ser_fwd_ctrl.tolist()
ls_fwd_exp =ser_fwd_exp.tolist()

ls_turn_ctrl =ser_turn_ctrl.tolist()
ls_turn_exp =ser_turn_exp.tolist()

In [35]:
ls_bwd_exp

[0.0, 0.0, 2.0, 0.0]

pingouinで検定をしていきます．

まずは対応のある/ないt検定から行ってみます．

[公式ドキュメント](https://pingouin-stats.org/build/html/generated/pingouin.ttest.html#pingouin.ttest)によると，引数は*array_like*と指定されています．
*array_like*とは何ぞや？と言いたいところですが，「どうやら`np.array()`関数に引き渡して`ndarray`が作れるもの」というのが[公式の回答のようです](https://numpy.org/doc/stable/user/basics.creation.html)．
`list`や`tuple`の形にしておけばとりあえず問題はなさそうです．

先ほど作成したリストをそのまま引数に与えることができます．

In [43]:
#paired t-test (bwd)
pg.ttest(ls_bwd_ctrl, ls_bwd_exp, paired=True)

c:\Users\zucky\anaconda3\Lib\site-packages\scipy\stats\_continuous_distns.py:7321: RuntimeWarning: divide by zero encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
c:\Users\zucky\anaconda3\Lib\site-packages\scipy\stats\_continuous_distns.py:7313: RuntimeWarning: divide by zero encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.0,3,two-sided,0.391002,"[-2.09, 1.09]",0.707107,0.613,0.172093


In [42]:
#unpaired t-test (bwd)
pg.ttest(ls_bwd_ctrl, ls_bwd_exp)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.0,6,two-sided,0.355918,"[-1.72, 0.72]",0.707107,0.695,0.135611


In [45]:
#paired t-test (fwd)
pg.ttest(ls_fwd_ctrl, ls_fwd_exp, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.080181,3,two-sided,0.12898,"[-4.37, 20.87]",1.894759,1.251,0.713814


In [46]:
#unpaired t-test (fwd)
pg.ttest(ls_fwd_ctrl, ls_fwd_exp)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.679593,6,two-sided,0.036556,"[0.72, 15.78]",1.894759,2.303,0.611154


In [47]:
#paired t-test (turn)
pg.ttest(ls_turn_ctrl, ls_turn_exp, paired=True)

c:\Users\zucky\anaconda3\Lib\site-packages\scipy\stats\_continuous_distns.py:7321: RuntimeWarning: divide by zero encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
c:\Users\zucky\anaconda3\Lib\site-packages\scipy\stats\_continuous_distns.py:7313: RuntimeWarning: divide by zero encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.127469,3,two-sided,0.341576,"[-2.28, 4.78]",1.035098,0.664,0.304661


In [51]:
#unpaired t-test (turn)
pg.ttest(ls_turn_ctrl, ls_turn_exp)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.46385,6,two-sided,0.19357,"[-0.84, 3.34]",1.035098,0.92,0.235635


2群の分散が等分散性が仮定できない場合に用いるWelchのt検定も行うことができます．

デフォルトではサンプルサイズが異なる場合のみにWelchのt検定を用いるようになっているため引数で指定して実行させます．

In [49]:
#unpaired t-test (turn)
pg.ttest(ls_turn_ctrl, ls_turn_exp, correction=False)

#correction=Welch's correctionであり，サンプルサイズが同じ場合には適用されていない

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.46385,6,two-sided,0.19357,"[-0.84, 3.34]",1.035098,0.92,0.235635


In [50]:
#Welch's t-test (turn)
pg.ttest(ls_turn_ctrl, ls_turn_exp, correction=True)

#correction=True とすることで補正を行わせる

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.46385,4.6343,two-sided,0.207564,"[-1.0, 3.5]",1.035098,0.92,0.235635


ちなみに，`ttest(~).round([int])`で返り値を小数第何位まで表示させるか(どこで丸めるか)を指定できます．

In [44]:
#paired t-test (bwd)
#Round an array to the given number of decimals
pg.ttest(ls_bwd_ctrl, ls_bwd_exp, paired=True).round(2)

c:\Users\zucky\anaconda3\Lib\site-packages\scipy\stats\_continuous_distns.py:7321: RuntimeWarning: divide by zero encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
c:\Users\zucky\anaconda3\Lib\site-packages\scipy\stats\_continuous_distns.py:7313: RuntimeWarning: divide by zero encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)


,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.0,3,two-sided,0.39,"[-2.09, 1.09]",0.71,0.613,0.17


次はグラフを描いていきます．

まずは描画に必要なものをimportします．

In [52]:
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import seaborn.objects as so

pingouinの個人的に感動したポイントはpaired plotが楽に描けることです．
[公式](https://pingouin-stats.org/build/html/generated/pingouin.plot_paired.html)によるとdataframeの形で与える必要があるようです．